In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('traffic_data.csv')

# Display the first few rows of the DataFrame
df.head()

,Roadblock,Time,Weather,Road Infrastructure,Types of Roads,Traffic Volume,Severity of Traffic
0,full-blockage,night,snow/storm,good,local,1000+,very-high
1,full-blockage,morning,light-rain,bad,local,500-749,very-high
2,full-blockage,morning,foggy,good,freeway,500-749,medium
3,partial-roadwork,early-morning,light-rain,bad,local,500-749,high
4,minor-accident,evening,foggy,good,collector,500-749,medium


In [12]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

df = pd.read_csv('traffic_data.csv')

# Convert categorical data to numerical using one-hot encoding
df_encoded = pd.get_dummies(df)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_encoded)

X = []
y = []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
24/24 [==============================] - 4s 50ms/step - loss: 0.1271 - val_loss: 0.1452
Epoch 2/100
24/24 [==============================] - 1s 25ms/step - loss: 0.1256 - val_loss: 0.1413
Epoch 3/100
24/24 [==============================] - 1s 27ms/step - loss: 0.1259 - val_loss: 0.1457
Epoch 4/100
24/24 [==============================] - 1s 25ms/step - loss: 0.1254 - val_loss: 0.1421
Epoch 5/100
24/24 [==============================] - 1s 25ms/step - loss: 0.1253 - val_loss: 0.1432
